In [1]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import re
import numpy as np

# File paths
input_file_path = r"C:\Users\polpi\Desktop\data science\project\docker_project\cleaned_formatted_excel.xlsx"
output_file_path = r"C:\Users\polpi\Desktop\data science\project\docker_project\updated_extracted_data2.xlsx"

# Column names
column_name = "formatted_info"
keyword_category_column = "Keyword_Category"

# Function to extract content between specified texts
def extract_info(text):
    patterns = {
        'Name': r'NAME\s*(.*?)\s*(?=AGE)',  # Extract everything between NAME and AGE
        'Age': r'AGE\s*(.*?)\s*(?=QUALIFICATION)',  # Extract everything between AGE and QUALIFICATION
        'Qualification': r'QUALIFICATION\s*(.*?)\s*(?=SUBJECT)',  # Extract everything between QUALIFICATION and SUBJECT
        'Subject Area': r'SUBJECT\s*(.*?)\s*(?=PLACE)',  # Extract everything between SUBJECT and PLACE
        'Place of Education': r'PLACE\s*(.*?)\s*(?=CODING)',  # Extract everything between PLACE and CODING
        'Coding Language': r'CODING\s*(.*?)\s*(?=SPOKEN)',  # Extract everything between CODING and SPOKEN
        'Spoken Language': r'SPOKEN\s*(.*?)\s*(?=SKILL)',  # Extract everything between SPOKEN and SKILL
        'Skill Set': r'SKILL\s*(.*?)\s*(?=EXPERIENCE)',  # Extract everything between SKILL and EXPERIENCE
        'Years of Work Experience': r'EXPERIENCE\s*(.*?)\s*(?=\s|$)'  # Extract everything between EXPERIENCE and end of string
    }
    extracted = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.DOTALL)
        value = match.group(1).strip() if match else np.nan
        extracted[key] = value
    return extracted

# Read the input Excel file
df_input = pd.read_excel(input_file_path)

# Process each row in the specified column
structured_data = [
    extract_info(cell) if isinstance(cell, str) and cell.strip() else {}
    for cell in df_input[column_name]
]

# Convert to DataFrame
df_output = pd.DataFrame(structured_data)

# Add Keyword_category column after "Years of Work Experience"
if keyword_category_column in df_input.columns:
    df_output.insert(
        df_output.columns.get_loc("Years of Work Experience") + 1,
        keyword_category_column,
        df_input[keyword_category_column]
    )

# Ensure Age and Years of Work Experience columns contain only numeric or NA values
def clean_numeric(column):
    return column.apply(lambda x: 'NA' if isinstance(x, (int, float, str)) and re.match(r'^\d+\.?\d*$', str(x)) and float(x) > 100 
                        else float(x) if isinstance(x, (int, float, str)) and re.match(r'^\d+\.?\d*$', str(x)) 
                        else 'NA' if pd.isna(x) or str(x).upper() == 'NA' else np.nan)

df_output['Age'] = clean_numeric(df_output['Age'])
df_output['Years of Work Experience'] = clean_numeric(df_output['Years of Work Experience'])


# Save the output to Excel
df_output.to_excel(output_file_path, index=False)

print("Data extraction completed successfully!")